# CosyVoice with Python 3.8

Building:

```sh
docker build -t funaudiollm_cp38 .
```


Running:

```sh
docker run -v $(pwd):/home/jovyan/work  -p 8888:8888 -it funaudiollm_cp38 
```

In [1]:
import re

# import gradio as gr
import torch
import os
from http import HTTPStatus

from typing import List, Optional, Tuple, Dict
from uuid import uuid4
from modelscope import HubApi
import torchaudio
import sys
sys.path.insert(1, "../cosyvoice")
sys.path.insert(1, "../sensevoice")
sys.path.insert(1, "../cosyvoice/third_party/AcademiCodec")
sys.path.insert(1, "../cosyvoice/third_party/Matcha-TTS")
sys.path.insert(1, "../")
from utils.rich_format_small import format_str_v2
from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
from funasr import AutoModel

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
2024-07-16 18:01:31,400 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2024-07-16 18:01:31,404 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-07-16 18:01:31,556 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 d15cd5768f4968c94214dd6357f645b9 and a total number of 980 components indexed


transformer is not installed, please install it if you want to use related modules
Notice: ffmpeg is not installed. torchaudio is used to load audio
If you want to use ffmpeg backend to load audio, please install it by:
	sudo apt install ffmpeg # ubuntu
	# brew install ffmpeg # mac


OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [2]:
from modelscope import snapshot_download
# snapshot_download('iic/CosyVoice-300M', local_dir='pretrained_models/CosyVoice-300M')
# snapshot_download('iic/CosyVoice-300M-SFT', local_dir='pretrained_models/CosyVoice-300M-SFT')
snapshot_download('iic/CosyVoice-300M-Instruct', local_dir='pretrained_models/CosyVoice-300M-Instruct')
# snapshot_download('iic/speech_kantts_ttsfrd', local_dir='pretrained_models/speech_kantts_ttsfrd')


Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27.0M/27.0M [01:03<00:00, 446kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47.0/47.0 [00:00<00:00, 48.4B/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.13k/6.13k [00:01<00:00, 5.61kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400M/400M [17:13<00:00, 406kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78.1M/78.1M [03:21<00:00, 406kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.16G/1.16G [13:

'pretrained_models/CosyVoice-300M-Instruct'

In [3]:
speaker_name = '中文女'
cosyvoice = CosyVoice('speech_tts/CosyVoice-300M-Instruct')
asr_model_name_or_path = "iic/SenseVoiceSmall"
sense_voice_model = AutoModel(model=asr_model_name_or_path,
                  vad_model="fsmn-vad",
                  vad_kwargs={"max_single_segment_time": 30000},
                  trust_remote_code=True, device="cuda:0", remote_code="./sensevoice/model.py")


Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27.0M/27.0M [01:07<00:00, 417kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47.0/47.0 [00:00<00:00, 51.7B/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6.13k/6.13k [00:01<00:00, 5.86kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400M/400M [15:08<00:00, 462kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78.1M/78.1M [02:16<00:00, 599kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.16G/1.16G [12:

AssertionError: failed to initialize ttsfrd resource

# Ugh, requires CUDA

In [ ]:
cosyvoice = CosyVoice('speech_tts/CosyVoice-300M-Instruct')
# instruct usage
output = cosyvoice.inference_instruct('在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。', '中文男', 'Theo \'Crimson\', is a fiery, passionate rebel leader. Fights with fervor for justice, but struggles with impulsiveness.')
torchaudio.save('instruct.wav', output['tts_speech'], 22050)

In [ ]:
cosyvoice = CosyVoice('speech_tts/CosyVoice-300M-SFT')
# sft usage
print(cosyvoice.list_avaliable_spks())
output = cosyvoice.inference_sft('你好，我是通义生成式语音大模型，请问有什么可以帮您的吗？', '中文女')
torchaudio.save('sft.wav', output['tts_speech'], 22050)